In [1]:
# Definir variables de entorno necesarias para GCS y el bucket
import os

os.environ["DATA_BUCKET"] = "energia-tfm-bucket"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "C:/Users/Administrador/Documents/TFM/tfm-energy-ingest/tfm-energia-streamlit-cloud-814352512664.json"
)
print("Variables de entorno definidas para DATA_BUCKET y GOOGLE_APPLICATION_CREDENTIALS.")

Variables de entorno definidas para DATA_BUCKET y GOOGLE_APPLICATION_CREDENTIALS.


In [2]:
import io
import os
import warnings
from typing import List, Optional

import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from google.cloud import storage

warnings.filterwarnings("ignore")


class GCSFileViewer:
    """
    Clase para visualizar y analizar archivos CSV y Parquet desde Google Cloud Storage
    """

    def __init__(self, project_id: Optional[str] = None, credentials_path: Optional[str] = None):
        """
        Inicializar el cliente GCS

        Args:
            project_id: ID del proyecto GCP (opcional)
            credentials_path: Ruta al archivo de credenciales (opcional)
        """
        if credentials_path:
            os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path

        self.client = storage.Client(project=project_id)
        print("✅ Cliente GCS inicializado correctamente")

    def list_files(self, bucket_name: str, prefix: str = "", limit: int = 100) -> List[str]:
        """
        Listar archivos en un bucket

        Args:
            bucket_name: Nombre del bucket
            prefix: Prefijo para filtrar archivos
            limit: Número máximo de archivos a listar

        Returns:
            Lista de nombres de archivos
        """
        bucket = self.client.bucket(bucket_name)
        blobs = bucket.list_blobs(prefix=prefix, max_results=limit)

        files = []
        print(f"📁 Archivos en gs://{bucket_name}/{prefix}:")
        print("-" * 80)

        for blob in blobs:
            size_mb = blob.size / (1024 * 1024) if blob.size else 0
            print(f"📄 {blob.name} ({size_mb:.2f} MB)")
            files.append(blob.name)

        print(f"\n🔍 Total: {len(files)} archivos encontrados")
        return files

    def read_file(self, bucket_name: str, file_path: str, **kwargs) -> pd.DataFrame:
        """
        Leer archivo CSV o Parquet desde GCS

        Args:
            bucket_name: Nombre del bucket
            file_path: Ruta del archivo
            **kwargs: Argumentos adicionales para pd.read_csv o pd.read_parquet

        Returns:
            DataFrame de pandas
        """
        try:
            # Determinar tipo de archivo por extensión
            file_extension = file_path.lower().split(".")[-1]

            if file_extension in ["csv", "txt"]:
                return self._read_csv(bucket_name, file_path, **kwargs)
            elif file_extension in ["parquet", "parque"]:
                return self._read_parquet(bucket_name, file_path, **kwargs)
            else:
                # Intentar como CSV por defecto
                print(f"⚠️  Extensión '{file_extension}' no reconocida. Intentando como CSV...")
                return self._read_csv(bucket_name, file_path, **kwargs)

        except Exception as e:
            print(f"❌ Error leyendo archivo: {str(e)}")
            return pd.DataFrame()

    def _read_csv(self, bucket_name: str, file_path: str, **kwargs) -> pd.DataFrame:
        """Leer archivo CSV"""
        bucket = self.client.bucket(bucket_name)
        blob = bucket.blob(file_path)

        # Descargar como string
        content = blob.download_as_text()

        # Leer con pandas
        df = pd.read_csv(io.StringIO(content), **kwargs)
        print(f"✅ CSV leído: {df.shape[0]} filas, {df.shape[1]} columnas")
        return df

    def _read_parquet(self, bucket_name: str, file_path: str, **kwargs) -> pd.DataFrame:
        """Leer archivo Parquet con manejo de errores de esquema"""
        gcs_path = f"gs://{bucket_name}/{file_path}"

        try:
            # Método 1: Pandas directo
            df = pd.read_parquet(gcs_path, **kwargs)
            print(f"✅ Parquet leído: {df.shape[0]} filas, {df.shape[1]} columnas")
            return df

        except Exception as e1:
            print(f"⚠️  Error con pandas directo: {str(e1)}")
            print("🔄 Intentando con PyArrow...")

            try:
                # Método 2: PyArrow con opciones compatibles
                import pyarrow.parquet as pq

                # Descargar archivo temporalmente
                bucket = self.client.bucket(bucket_name)
                blob = bucket.blob(file_path)

                # Leer bytes y usar PyArrow
                parquet_bytes = blob.download_as_bytes()
                table = pq.read_table(
                    io.BytesIO(parquet_bytes),
                    use_pandas_metadata=False,  # Ignorar metadatos pandas
                    **kwargs,
                )
                df = table.to_pandas()
                print(f"✅ Parquet leído con PyArrow: {df.shape[0]} filas, {df.shape[1]} columnas")
                return df

            except Exception as e2:
                print(f"⚠️  Error con PyArrow: {str(e2)}")
                print("🔄 Intentando con esquema flexible...")

                try:
                    # Método 3: Leer con esquema más flexible
                    import pyarrow.parquet as pq

                    parquet_bytes = blob.download_as_bytes()
                    parquet_file = pq.ParquetFile(io.BytesIO(parquet_bytes))

                    # Leer por chunks para evitar conflictos de esquema
                    dfs = []
                    for batch in parquet_file.iter_batches(batch_size=1000):
                        chunk_df = batch.to_pandas()
                        # Convertir columnas categóricas a string
                        for col in chunk_df.columns:
                            if chunk_df[col].dtype.name == "category":
                                chunk_df[col] = chunk_df[col].astype(str)
                            elif pd.api.types.is_categorical_dtype(chunk_df[col]):
                                chunk_df[col] = chunk_df[col].astype(str)
                        dfs.append(chunk_df)

                    if dfs:
                        df = pd.concat(dfs, ignore_index=True)
                        print(f"✅ Parquet leído con esquema flexible: {df.shape[0]} filas, {df.shape[1]} columnas")
                        return df
                    else:
                        print("❌ No se pudieron leer datos del archivo")
                        return pd.DataFrame()

                except Exception as e3:
                    print(f"❌ Error final: {str(e3)}")
                    print("💡 Sugerencia: El archivo podría estar corrupto o tener un esquema muy complejo")
                    return pd.DataFrame()

    def preview_file(self, bucket_name: str, file_path: str, n_rows: int = 10) -> None:
        """
        Vista previa rápida del archivo

        Args:
            bucket_name: Nombre del bucket
            file_path: Ruta del archivo
            n_rows: Número de filas a mostrar
        """
        print(f"👀 Vista previa de: gs://{bucket_name}/{file_path}")
        print("=" * 80)

        df = self.read_file(bucket_name, file_path)
        if df.empty:
            return

        # Información básica
        print(f"📊 Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")
        print(f"💾 Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        print()

        # Mostrar primeras filas
        print(f"🔝 Primeras {min(n_rows, len(df))} filas:")
        print("-" * 80)
        display(df.head(n_rows))

        # Información de columnas
        print("\n📋 Información de columnas:")
        print("-" * 80)
        display(df.info())

        # Estadísticas básicas para columnas numéricas
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0:
            print(f"\n📈 Estadísticas básicas ({len(numeric_cols)} columnas numéricas):")
            print("-" * 80)
            display(df[numeric_cols].describe())

    def diagnose_parquet(self, bucket_name: str, file_path: str) -> dict:
        """
        Diagnosticar problemas en archivo Parquet

        Args:
            bucket_name: Nombre del bucket
            file_path: Ruta del archivo

        Returns:
            Diccionario con información de diagnóstico
        """
        try:
            bucket = self.client.bucket(bucket_name)
            blob = bucket.blob(file_path)
            parquet_bytes = blob.download_as_bytes()
            parquet_file = pq.ParquetFile(io.BytesIO(parquet_bytes))

            schema = parquet_file.schema_arrow
            metadata = parquet_file.metadata

            diagnosis = {
                "schema": str(schema),
                "num_columns": schema.num_columns,
                "num_rows": metadata.num_rows,
                "num_row_groups": metadata.num_row_groups,
                "file_size_bytes": len(parquet_bytes),
                "column_info": {},
            }

            # Información detallada de columnas
            for i, field in enumerate(schema):
                diagnosis["column_info"][field.name] = {"type": str(field.type), "nullable": field.nullable}

            print(f"🔍 Diagnóstico de: gs://{bucket_name}/{file_path}")
            print("=" * 80)
            print(f"📊 Filas: {diagnosis['num_rows']:,}")
            print(f"📋 Columnas: {diagnosis['num_columns']}")
            print(f"📦 Row Groups: {diagnosis['num_row_groups']}")
            print(f"💾 Tamaño: {diagnosis['file_size_bytes']:,} bytes")

            print("\n📝 Esquema de columnas:")
            print("-" * 80)
            for col, info in diagnosis["column_info"].items():
                print(f"  • {col}: {info['type']}")

            return diagnosis

        except Exception as e:
            print(f"❌ Error en diagnóstico: {str(e)}")
            return {}
        """
        Análisis completo del archivo
        
        Args:
            bucket_name: Nombre del bucket
            file_path: Ruta del archivo
        
        Returns:
            Diccionario con métricas del análisis
        """
        df = self.read_file(bucket_name, file_path)
        if df.empty:
            return {}

        analysis = {
            "shape": df.shape,
            "memory_usage_mb": df.memory_usage(deep=True).sum() / 1024**2,
            "dtypes": df.dtypes.to_dict(),
            "null_counts": df.isnull().sum().to_dict(),
            "null_percentages": (df.isnull().sum() / len(df) * 100).to_dict(),
            "numeric_columns": list(df.select_dtypes(include=[np.number]).columns),
            "categorical_columns": list(df.select_dtypes(include=["object"]).columns),
            "datetime_columns": list(df.select_dtypes(include=["datetime64"]).columns),
        }

        # Mostrar análisis
        print(f"🔍 Análisis completo de: gs://{bucket_name}/{file_path}")
        print("=" * 80)
        print(f"📐 Dimensiones: {analysis['shape'][0]:,} filas × {analysis['shape'][1]} columnas")
        print(f"💾 Uso de memoria: {analysis['memory_usage_mb']:.2f} MB")
        print(f"🔢 Columnas numéricas: {len(analysis['numeric_columns'])}")
        print(f"📝 Columnas categóricas: {len(analysis['categorical_columns'])}")
        print(f"📅 Columnas datetime: {len(analysis['datetime_columns'])}")

        # Valores nulos
        null_cols = [col for col, pct in analysis["null_percentages"].items() if pct > 0]
        if null_cols:
            print(f"\n⚠️  Columnas con valores nulos ({len(null_cols)}):")
            for col in null_cols[:10]:  # Mostrar solo las primeras 10
                pct = analysis["null_percentages"][col]
                print(f"   • {col}: {pct:.1f}%")
            if len(null_cols) > 10:
                print(f"   ... y {len(null_cols) - 10} más")

        return analysis


# =============================================================================
# FUNCIONES DE UTILIDAD
# =============================================================================


def quick_preview(bucket_name: str, file_path: str, n_rows: int = 5):
    """Vista previa rápida - función standalone"""
    viewer = GCSFileViewer()
    viewer.preview_file(bucket_name, file_path, n_rows)


def list_bucket_files(bucket_name: str, prefix: str = "", limit: int = 50):
    """Listar archivos - función standalone"""
    viewer = GCSFileViewer()
    return viewer.list_files(bucket_name, prefix, limit)


def diagnose_parquet_file(bucket_name: str, file_path: str):
    """Diagnosticar archivo Parquet - función standalone"""
    viewer = GCSFileViewer()
    return viewer.diagnose_parquet(bucket_name, file_path)


def load_gcs_file(bucket_name: str, file_path: str, **kwargs) -> pd.DataFrame:
    """Cargar archivo - función standalone"""
    viewer = GCSFileViewer()
    return viewer.read_file(bucket_name, file_path, **kwargs)


# =============================================================================
# EJEMPLOS DE USO
# =============================================================================

if __name__ == "__main__":
    # Configuración
    BUCKET_NAME = "energia-tfm-bucket"

    # Inicializar viewer
    viewer = GCSFileViewer()

    # Ejemplo 1: Listar archivos
    print("🚀 EJEMPLO 1: Listar archivos")
    files = viewer.list_files(BUCKET_NAME, prefix="curated/prices/", limit=10)

    # Ejemplo 2: Vista previa rápida
    if files:
        print("\n🚀 EJEMPLO 2: Vista previa del primer archivo")
        viewer.preview_file(BUCKET_NAME, files[0], n_rows=5)

    # Ejemplo 3: Cargar archivo completo
    if files:
        print("\n🚀 EJEMPLO 3: Cargar archivo completo")
        df = viewer.read_file(BUCKET_NAME, files[0])
        print(f"DataFrame cargado: {df.shape}")

# =============================================================================
# INSTRUCCIONES DE USO
# =============================================================================
"""
📖 GUÍA DE USO:

1. CONFIGURACIÓN INICIAL:
   viewer = GCSFileViewer()
   # O con credenciales específicas:
   viewer = GCSFileViewer(project_id="tu-proyecto", credentials_path="path/to/creds.json")

2. LISTAR ARCHIVOS:
   files = viewer.list_files("tu-bucket", prefix="carpeta/", limit=20)

3. VISTA PREVIA RÁPIDA:
   viewer.preview_file("tu-bucket", "archivo.csv", n_rows=10)

4. CARGAR ARCHIVO COMPLETO:
   df = viewer.read_file("tu-bucket", "archivo.parquet")

5. ANÁLISIS COMPLETO:
   analysis = viewer.analyze_file("tu-bucket", "archivo.csv")

6. FUNCIONES RÁPIDAS:
   quick_preview("bucket", "file.csv")
   df = load_gcs_file("bucket", "file.parquet")
   files = list_bucket_files("bucket", "prefix/")

🔧 PARÁMETROS ADICIONALES:
   # Para CSV
   df = viewer.read_file("bucket", "file.csv", sep=";", encoding="utf-8")
   
   # Para Parquet
   df = viewer.read_file("bucket", "file.parquet", columns=["col1", "col2"])
"""

✅ Cliente GCS inicializado correctamente
🚀 EJEMPLO 1: Listar archivos
📁 Archivos en gs://energia-tfm-bucket/curated/prices/:
--------------------------------------------------------------------------------
📄 curated/prices/year=2025/month=09/compact.parquet (0.00 MB)

🔍 Total: 1 archivos encontrados

🚀 EJEMPLO 2: Vista previa del primer archivo
👀 Vista previa de: gs://energia-tfm-bucket/curated/prices/year=2025/month=09/compact.parquet
✅ Parquet leído: 192 filas, 7 columnas
📊 Dimensiones: 192 filas × 7 columnas
💾 Memoria: 0.03 MB

🔝 Primeras 5 filas:
--------------------------------------------------------------------------------


,hour_ts,price_eur_mwh,zone,source,indicator_id,year,month
0,2025-09-09 00:00:00+02:00,106.76,España,PVPC,1001,2025,9
1,2025-09-09 00:00:00+02:00,114.52,Alemania,SPOT_ES,600,2025,9
2,2025-09-09 00:00:00+02:00,79.00,Bélgica,SPOT_ES,600,2025,9
3,2025-09-09 00:00:00+02:00,63.33,España,SPOT_ES,600,2025,9
4,2025-09-09 00:00:00+02:00,37.23,Francia,SPOT_ES,600,2025,9



📋 Información de columnas:
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype                        
---  ------         --------------  -----                        
 0   hour_ts        192 non-null    datetime64[ns, Europe/Madrid]
 1   price_eur_mwh  192 non-null    float64                      
 2   zone           192 non-null    object                       
 3   source         192 non-null    object                       
 4   indicator_id   192 non-null    int64                        
 5   year           192 non-null    category                     
 6   month          192 non-null    category                     
dtypes: category(2), datetime64[ns, Europe/Madrid](1), float64(1), int64(1), object(2)
memory usage: 8.2+ KB


None


📈 Estadísticas básicas (2 columnas numéricas):
--------------------------------------------------------------------------------


,price_eur_mwh,indicator_id
count,192.000000,192.000
mean,100.171823,650.125
std,58.740335,132.965
min,8.910000,600.000
25%,70.380000,600.000
50%,93.185000,600.000
75%,114.400000,600.000
max,386.400000,1001.000



🚀 EJEMPLO 3: Cargar archivo completo
✅ Parquet leído: 192 filas, 7 columnas
DataFrame cargado: (192, 7)


'\n📖 GUÍA DE USO:\n\n1. CONFIGURACIÓN INICIAL:\n   viewer = GCSFileViewer()\n   # O con credenciales específicas:\n   viewer = GCSFileViewer(project_id="tu-proyecto", credentials_path="path/to/creds.json")\n\n2. LISTAR ARCHIVOS:\n   files = viewer.list_files("tu-bucket", prefix="carpeta/", limit=20)\n\n3. VISTA PREVIA RÁPIDA:\n   viewer.preview_file("tu-bucket", "archivo.csv", n_rows=10)\n\n4. CARGAR ARCHIVO COMPLETO:\n   df = viewer.read_file("tu-bucket", "archivo.parquet")\n\n5. ANÁLISIS COMPLETO:\n   analysis = viewer.analyze_file("tu-bucket", "archivo.csv")\n\n6. FUNCIONES RÁPIDAS:\n   quick_preview("bucket", "file.csv")\n   df = load_gcs_file("bucket", "file.parquet")\n   files = list_bucket_files("bucket", "prefix/")\n\n🔧 PARÁMETROS ADICIONALES:\n   # Para CSV\n   df = viewer.read_file("bucket", "file.csv", sep=";", encoding="utf-8")\n\n   # Para Parquet\n   df = viewer.read_file("bucket", "file.parquet", columns=["col1", "col2"])\n'

In [7]:
# 1. Usar uno de los archivos listados anteriormente
files = viewer.list_files("energia-tfm-bucket", "curated/prices/", limit=10)
print("Archivos disponibles:", files)

# 2. Vista previa del primer archivo
if files:
    viewer.preview_file("energia-tfm-bucket", files[0])

# 3. O directamente (SIN gs://bucket-name/)
viewer.preview_file(
    "energia-tfm-bucket",
    "curated/prices/indicator_id=1001/zone=Baleares/year=2025/month=09/day=21/part-0360d6ce-c082-4fe5-8f52-08f0ecac9e54.parquet",
)

📁 Archivos en gs://energia-tfm-bucket/curated/prices/:
--------------------------------------------------------------------------------
📄 curated/prices/year=2025/month=09/compact.parquet (0.00 MB)

🔍 Total: 1 archivos encontrados
Archivos disponibles: ['curated/prices/year=2025/month=09/compact.parquet']
👀 Vista previa de: gs://energia-tfm-bucket/curated/prices/year=2025/month=09/compact.parquet
✅ Parquet leído: 24 filas, 7 columnas
📊 Dimensiones: 24 filas × 7 columnas
💾 Memoria: 0.00 MB

🔝 Primeras 10 filas:
--------------------------------------------------------------------------------


,hour_ts,price_eur_mwh,zone,source,indicator_id,year,month
0,2025-09-09 00:00:00+02:00,106.76,España,PVPC,1001,2025,9
1,2025-09-09 01:00:00+02:00,107.28,España,PVPC,1001,2025,9
2,2025-09-09 02:00:00+02:00,106.52,España,PVPC,1001,2025,9
3,2025-09-09 03:00:00+02:00,103.72,España,PVPC,1001,2025,9
4,2025-09-09 04:00:00+02:00,104.19,España,PVPC,1001,2025,9
5,2025-09-09 05:00:00+02:00,111.25,España,PVPC,1001,2025,9
6,2025-09-09 06:00:00+02:00,127.23,España,PVPC,1001,2025,9
7,2025-09-09 07:00:00+02:00,140.44,España,PVPC,1001,2025,9
8,2025-09-09 08:00:00+02:00,163.95,España,PVPC,1001,2025,9
9,2025-09-09 09:00:00+02:00,146.31,España,PVPC,1001,2025,9



📋 Información de columnas:
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype                        
---  ------         --------------  -----                        
 0   hour_ts        24 non-null     datetime64[ns, Europe/Madrid]
 1   price_eur_mwh  24 non-null     float64                      
 2   zone           24 non-null     object                       
 3   source         24 non-null     object                       
 4   indicator_id   24 non-null     int64                        
 5   year           24 non-null     category                     
 6   month          24 non-null     category                     
dtypes: category(2), datetime64[ns, Europe/Madrid](1), float64(1), int64(1), object(2)
memory usage: 1.3+ KB


None


📈 Estadísticas básicas (2 columnas numéricas):
--------------------------------------------------------------------------------


,price_eur_mwh,indicator_id
count,24.000000,24.0
mean,143.295833,1001.0
std,50.903536,0.0
min,70.500000,1001.0
25%,105.937500,1001.0
50%,142.395000,1001.0
75%,168.280000,1001.0
max,253.580000,1001.0


👀 Vista previa de: gs://energia-tfm-bucket/curated/prices/indicator_id=1001/zone=Baleares/year=2025/month=09/day=21/part-0360d6ce-c082-4fe5-8f52-08f0ecac9e54.parquet
⚠️  Error con pandas directo: energia-tfm-bucket/curated/prices/indicator_id=1001/zone=Baleares/year=2025/month=09/day=21/part-0360d6ce-c082-4fe5-8f52-08f0ecac9e54.parquet
🔄 Intentando con PyArrow...
⚠️  Error con PyArrow: 404 GET https://storage.googleapis.com/download/storage/v1/b/energia-tfm-bucket/o/curated%2Fprices%2Findicator_id%3D1001%2Fzone%3DBaleares%2Fyear%3D2025%2Fmonth%3D09%2Fday%3D21%2Fpart-0360d6ce-c082-4fe5-8f52-08f0ecac9e54.parquet?alt=media: No such object: energia-tfm-bucket/curated/prices/indicator_id=1001/zone=Baleares/year=2025/month=09/day=21/part-0360d6ce-c082-4fe5-8f52-08f0ecac9e54.parquet: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)
🔄 Intentando con esquema flexible...
❌ Error final: 404 GET https://storage.googleapis.com/down

In [8]:
nombre_fichero = "raw/prices_spot/year=2025/month=09/day=21/prices_spot_2025-09-21T10:13:14.814681Z.csv"

viewer.preview_file("energia-tfm-bucket", nombre_fichero)

nombre_fichero2 = "curated/prices/indicator_id=600/zone=Alemania/year=2025/month=09/day=21/part-941bb899-82bb-4e03-9642-721be1a05c5c.parquet"

viewer.preview_file("energia-tfm-bucket", nombre_fichero2)

👀 Vista previa de: gs://energia-tfm-bucket/raw/prices_spot/year=2025/month=09/day=21/prices_spot_2025-09-21T10:13:14.814681Z.csv
❌ Error leyendo archivo: 404 GET https://storage.googleapis.com/download/storage/v1/b/energia-tfm-bucket/o/raw%2Fprices_spot%2Fyear%3D2025%2Fmonth%3D09%2Fday%3D21%2Fprices_spot_2025-09-21T10%3A13%3A14.814681Z.csv?alt=media: No such object: energia-tfm-bucket/raw/prices_spot/year=2025/month=09/day=21/prices_spot_2025-09-21T10:13:14.814681Z.csv: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)
👀 Vista previa de: gs://energia-tfm-bucket/curated/prices/indicator_id=600/zone=Alemania/year=2025/month=09/day=21/part-941bb899-82bb-4e03-9642-721be1a05c5c.parquet
⚠️  Error con pandas directo: energia-tfm-bucket/curated/prices/indicator_id=600/zone=Alemania/year=2025/month=09/day=21/part-941bb899-82bb-4e03-9642-721be1a05c5c.parquet
🔄 Intentando con PyArrow...
⚠️  Error con PyArrow: 404 GET https://stora

In [9]:
nombre_fichero = "raw/prices_pvpc/year=2025/month=09/day=21/prices_pvpc_2025-09-21T10:19:31.780426Z.csv"

viewer.preview_file("energia-tfm-bucket", nombre_fichero)

nombre_fichero2 = "curated/prices/indicator_id=1001/zone=Península/year=2025/month=09/day=22/part-1ad813ae-40f9-43a0-821b-ad29d26bbe6c.parquet"
viewer.preview_file("energia-tfm-bucket", nombre_fichero2)

👀 Vista previa de: gs://energia-tfm-bucket/raw/prices_pvpc/year=2025/month=09/day=21/prices_pvpc_2025-09-21T10:19:31.780426Z.csv
❌ Error leyendo archivo: 404 GET https://storage.googleapis.com/download/storage/v1/b/energia-tfm-bucket/o/raw%2Fprices_pvpc%2Fyear%3D2025%2Fmonth%3D09%2Fday%3D21%2Fprices_pvpc_2025-09-21T10%3A19%3A31.780426Z.csv?alt=media: No such object: energia-tfm-bucket/raw/prices_pvpc/year=2025/month=09/day=21/prices_pvpc_2025-09-21T10:19:31.780426Z.csv: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)
👀 Vista previa de: gs://energia-tfm-bucket/curated/prices/indicator_id=1001/zone=Península/year=2025/month=09/day=22/part-1ad813ae-40f9-43a0-821b-ad29d26bbe6c.parquet
⚠️  Error con pandas directo: energia-tfm-bucket/curated/prices/indicator_id=1001/zone=Península/year=2025/month=09/day=22/part-1ad813ae-40f9-43a0-821b-ad29d26bbe6c.parquet
🔄 Intentando con PyArrow...
⚠️  Error con PyArrow: 404 GET https://s

In [5]:
# 4. CARGAR ARCHIVO COMPLETO:
archivo = "curated/prices/year=2025/month=03/day=10/part-02e294e3-843c-4d15-9564-d665d6721e52.parquet"

df = viewer.read_file("energia-tfm-bucket", archivo)

df

✅ Parquet leído: 24 filas, 8 columnas


,hour_ts,price_eur_mwh,zone,source,indicator_id,year,month,day
0,2025-03-10 00:00:00+01:00,120.05,España,PVPC,1001,2025,3,10
1,2025-03-10 01:00:00+01:00,113.42,España,PVPC,1001,2025,3,10
2,2025-03-10 02:00:00+01:00,109.62,España,PVPC,1001,2025,3,10
3,2025-03-10 03:00:00+01:00,109.06,España,PVPC,1001,2025,3,10
4,2025-03-10 04:00:00+01:00,103.58,España,PVPC,1001,2025,3,10
5,2025-03-10 05:00:00+01:00,109.80,España,PVPC,1001,2025,3,10
6,2025-03-10 06:00:00+01:00,136.12,España,PVPC,1001,2025,3,10
7,2025-03-10 07:00:00+01:00,161.17,España,PVPC,1001,2025,3,10
8,2025-03-10 08:00:00+01:00,192.50,España,PVPC,1001,2025,3,10
9,2025-03-10 09:00:00+01:00,148.62,España,PVPC,1001,2025,3,10
